In [8]:
#tkinter
import tkinter as tk
import tkinter.ttk as ttk
import threading
from PIL import ImageTk, Image
from functools import partial

#selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from urllib.parse import unquote

#other
import time
import json
import os
import import_ipynb

#show image
from IPython.core.debugger import set_trace
from IPython.display import Image as iImage
from IPython.core.display import HTML
from IPython.core.debugger import set_trace
from IPython.display import clear_output as clear
from urllib.request import urlopen
from io import BytesIO

class ListComic:
    def __init__(self,window,driver):
        self.driver = driver
        self.window = window
        self.window.configure(background = 'black')
        #self.main_frame = tk.Frame(self.window)
        
        #scrollable things
        self.main_canvas = tk.Canvas(self.window)
        self.main_canvas.pack(expand = True,side = 'left',fill = 'both')
        
        self.main_scrollbar = tk.Scrollbar(self.window,orient = 'vertical',command = self.main_canvas.yview)
        self.main_scrollbar.pack(side = 'left',fill = 'y')
        
        self.main_canvas.configure(yscrollcommand = self.main_scrollbar.set)
        self.main_canvas.bind('<Configure>',self.on_configure)
        self.main_canvas.bind_all('<MouseWheel>', self._on_mousewheel)
        
        #need to add width,height information for centering
        self.main_frame = tk.Frame(self.main_canvas,width = 500,height = 300)
        self.main_frame.configure(bg = 'black')
        self.main_frame.pack(expand = True,side = 'left',fill = 'both')
        self.main_canvas.configure(bg = 'black')
        self.main_canvas.create_window((0,0),window = self.main_frame,anchor = 'nw')
        
        self.bg = 'black'
        self.fg = 'orange'
        self.ft = 'Helvetica 12 bold'
        
        #manga cover list
        self.readable_manga_cover = []
        #manga title list
        self.readable_manga_title = []
        
        self.start_list_of_readable_manga()
    
    #scrollable things
    def on_configure(self,event):
        self.main_canvas.configure(scrollregion = self.main_canvas.bbox("all"))
        
    def _on_mousewheel(self, event):
        self.main_canvas.yview_scroll(int(-1*(event.delta/50)), "units")
        
    def clear_all_widgets(self):
        
        for widget in self.main_frame.winfo_children():
            widget.destroy()
        
    def list_all_in_frame(self):
#         self.main_frame.configure(bg = self.bg)
#         self.main_frame.pack(expand = True,fill = 'both')
        
        #show image 
        p = 0.2
        
        comics_title = []
        pre_cover = []
        comics = []
        
        comic_cnt = 0
        cnt = 0
        #row_cm = 0
        
#         for k in range(5):
#             comics_title.append(tk.Label(self.main_frame,text = 'jizz' + str(k + 1)))
#             comics_title[k].grid(row = row_cm + k,column = 0,columnspan = 3,sticky = 'news')
            
#             row_cm += 1
            
        for i in range(15):
            #row_cm += 1
            
            #manga category title
            if i % 3 == 0:
                comics.append(tk.Label(self.main_frame,text = 'jizz' + str(i // 3 + 1),bg = self.bg,fg = self.fg,font = self.ft))
                comics[i].grid(row = i,column = 0,columnspan = 3,sticky = 'news')
                continue
            
            #not title
            for j in range(2):
                if j == 0:
                    comics.append([])

                if i % 3 == 1:

                    #pre_cover.append(Image.open('mcover' + str(comic_cnt + 1) + '.jpg'))
                    image_url = urlopen(self.readable_manga_cover[comic_cnt])
                    url_data = image_url.read()
                    image_url.close()
                    pre_cover.append(Image.open(BytesIO(url_data)))
                    
                    pre_w , pre_h = pre_cover[comic_cnt].size[0],pre_cover[comic_cnt].size[1]
                    pre_cover[comic_cnt] = pre_cover[comic_cnt].resize((int(p * pre_w),int(p * pre_h)),Image.ANTIALIAS)

                    cover = ImageTk.PhotoImage(pre_cover[comic_cnt])
                    comics[i].append(tk.Label(self.main_frame,image = cover,bg = self.bg))
                    comics[i][j].photo = cover
                    #comics[i].pack(expand = True,fill = 'both')
                    comics[i][j].grid(row = i,column = j,sticky = 'news')

                    comic_cnt += 1

                else :
                    
                    l = tk.Button(self.main_frame,text = self.readable_manga_title[cnt],bg = self.bg,fg = self.fg,font = "Helvetica 8 bold",command = None)

                    comics[i].append(l)

                    comics[i][j].grid(row = i,column = j,sticky = 'news')

                    cnt += 1
        
    # webdriver
    
    def add_login_cookie(self):
        fc = open('comicFuz.json')
        cookie = fc.read()
        cookie = json.loads(cookie)

        for c in cookie:
            if 'expiry' in c:
                del c['expiry']
            self.driver.add_cookie(c)

        self.driver.refresh()
        
    def check_thread_alive(self,thread):
        if thread.is_alive():
            self.window.after(20,self.check_thread_alive,thread)
        else :
            pass
        
    def start_list_of_readable_manga(self):
        
        loading_info = tk.Label(self.main_frame,text = "Next page is loading...",bg = self.bg,fg = self.fg,font = self.ft,height = 2)
        loading_info.place(relx = .47,rely = .4,anchor = 'c')
        #progressbar
        progress_bar = ttk.Progressbar(self.main_frame,orient='horizontal',length = 100,mode = 'determinate')
        progress_bar.place(relx = .47,rely = .5,anchor = 'c')
        
        progress_bar['value'] = 0
        self.window.update_idletasks()
        
        th = threading.Thread(target = self.list_of_readable_manga,args = (progress_bar,loading_info,))
        th.setDaemon(True)
        th.start()
        self.window.after(20,self.check_thread_alive,th)
    
    def list_of_readable_manga(self,progress_bar,loading_info):
        
        piece = 100 / 22
        loading = 0
        
        url = 'https://comic-fuz.com/series/'
        
        self.driver.get(url)
        
        self.add_login_cookie()
        
        #number(id) of cover
        cnt = 1
        readable_link_list = []

        for series_num in range(1,6):

            if url[-1] == '/':
                url += str(series_num)
            else :
                url = url[:len(url) - 1:]
                url += str(series_num)

            print(url)
            self.driver.get(url)

            time.sleep(2)

            xpath = '//*[contains(@class,"list")]'
            comic_table = self.driver.find_element_by_xpath(xpath)
            #print(comic_table.get_attribute("class"))
            comic_list = comic_table.find_elements_by_tag_name("li")

            series_cnt = 0

            for item in comic_list:
                #check which li is it
                series_cnt += 1
                #check if it is rented
                if '読む' in item.text:
                    print(cnt," : ")
                    cnt += 1
                    
                    if cnt != 23:
                        progress_bar['value'] = loading + piece
                        loading += piece
                        self.window.update_idletasks()
                    else :
                        progress_bar['value'] = 100
                        self.window.update_idletasks()

                    title = item.find_element_by_class_name("title")
                    self.readable_manga_title.append(title.text)
                    print(title.text)

                    link_path = '/html/body/main/div/div/div/section/div/ul/li[' + str(series_cnt) + ']/div[2]/div[1]/div/a'
                    link = item.find_element_by_xpath(link_path)
                    readable_link_list.append(link.get_attribute("href"))

                    rimg = item.find_element_by_tag_name("img").get_attribute("src")
                    self.readable_manga_cover.append(rimg)
                    display(iImage(url = rimg,width = 150,height = 215))
            
            print('finish')
        
        time.sleep(2)
        
        progress_bar.destroy()
        loading_info.destroy()
        
        self.list_all_in_frame()
            
window = tk.Tk()
window.title('List')
window.geometry('500x300')
window.resizable(0,0)

chrome_options = Options()
chrome_options.add_argument('headless')
driver = webdriver.Chrome(options = chrome_options)

#setting new icon
icon = ImageTk.PhotoImage(file = 'Cover.png')
window.tk.call('wm','iconphoto',window._w,icon)
print(icon)

page = ListComic(window,driver)

window.mainloop()

pyimage50
https://comic-fuz.com/series/1
https://comic-fuz.com/series/1
